<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# Get helper functions

In [9]:
!wget https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py

--2025-05-20 14:49:32--  https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10129 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.89K  --.-KB/s    in 0s      

2025-05-20 14:49:32 (52.4 MB/s) - ‘helper_functions.py’ saved [10129/10129]



In [10]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

## Get a text dataset

In [11]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2025-05-20 14:49:33--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.207, 74.125.199.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2025-05-20 14:49:33 (77.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing data
with the help of pandas


In [12]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
# suffling train data
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [15]:
# how many samples?
len(train_df),len(test_df)

(7613, 3263)

In [16]:
# Lets visualize some random samples.
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","real disaster" if target>0 else "not real disaster")
  print(f"Text: {text}")
  print("_____")

Target:1 real disaster
Text: Cop pulls drunk driver to safety SECONDS before his car is hit by train. http://t.co/tHrhKHOGcUåÊ http://t.co/tZSZmF2qxE via @ViralSpell
_____
Target:1 real disaster
Text: Cyclone Komen devastates families in Myanmar this week. We need to help them today: http://t.co/fCujsIOyQO
_____
Target:1 real disaster
Text: I added a video to a @YouTube playlist http://t.co/bcjYleRRYX Ori and the Bind forest ep 6 'Fire and death'
_____
Target:0 not real disaster
Text: TWIA board approves 5 percent rate hike: The Texas Windstorm Insurance Association (TWIA) Board of Directors v... http://t.co/esEMjRn5cC
_____
Target:0 not real disaster
Text: It's a deluge in Trois-Rivieres. About one hour to get to #legionstrackandfield http://t.co/PuE5xNZnQB
_____


In [17]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## split data in to training and validation set

In [18]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1, # use 10% of training data for validation
                                                                       random_state=42)

In [19]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [20]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text vectorizarion (Tokenization)
### Converting text into numbers

*   Tokenization: Tokenization is the process of breaking down text into smaller units called tokens, which can be words, subwords, or even characters. Its a direct mapping of token to a number
*   Embedding: Create a matrix of feature vector for each token.
Embeddings convert tokens (words/subwords) into numerical vectors so that machines can process them. They capture semantic meaning (e.g., "king" is closer to "queen" than to "apple").Contextual Embeddings (Modern NLP)

Words get different vectors based on context (e.g., BERT, GPT).



In [21]:
import tensorflow as tf
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=None,
                                                  standardize='lower_and_strip_punctuation',
                                                  split='whitespace',
                                                  ngrams=None, #create groups of n-words
                                                  output_mode='int',
                                                  output_sequence_length=None, # how long do u want your sequence to be?
                                                  pad_to_max_tokens=False
                                                  )


### find the average number of tokens or words in training samples

In [22]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [23]:
# setup text vectorization variables
max_vocab_length=10000 # Maximum number of words to have in our vocabolary
max_length=15 #max length our sentences will be
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode='int',
                                                  output_sequence_length=max_length)

In [24]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [25]:
# lets create a sample sentences and tokenize it
sample_sentence="The recent earthquake has destroyed the entire city"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2756,  290,   41,  351,    2,  855,  182,    0,    0,    0,
           0,    0,    0,    0]])>

In [26]:
# selecting a random sample from the train dataset and tokenize it
import random
random_sentence=random.choice(train_sentences)
print(f"Random Sample: {random_sentence}")
print(f"Vectorized sample: {text_vectorizer([random_sentence])}")

Random Sample: During the 1960s the oryx a symbol of the Arabian Peninsula were annihilated by hunters. 
http://t.co/yangEQBUQW http://t.co/jQ2eH5KGLt
Vectorized sample: [[ 246    2    1    2    1    3 7848    6    2 3178 9867   67  558   18
  2089]]


In [27]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # Get all the unique words in the training dataset
top_5_words=words_in_vocab[:5] # get the top 5 most common words
bottom_5_words=words_in_vocab[-5:] # get the least 5 common words
print(f"Words in Vocab: {len(words_in_vocab)}")
print(f"Top 5 words: {top_5_words}")
print(f"Least 5 words: {bottom_5_words}")

Words in Vocab: 10000
Top 5 words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Least 5 words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### creating an Embedding using Embedding layer

*   input_dim=the size of our vocabolary
*   output_dim= the size of the output embedding vector
*   input_length= size of the sequences being passed to the embedding layer

Key Properties of an Embedding Layer

**Trainable:**

* The vectors are updated via backpropagation (like other weights in a neural network).

* Pretrained embeddings (e.g., GloVe) can also be fine-tuned.

**Efficiency:**
More efficient than one-hot encoding + dense layers (avokes high-dimensional sparse matrices).

**Interpretability:**
Learned embeddings often capture semantic relationships (e.g., similar words cluster together).


In [28]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, # output shape
                           input_length=max_length # how long is each input
                           )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
# embed the random sample. i.e turn into dense vector of fixed size
print("Random sample: ",random_sentence)
print("After Embedding:...................")
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Random sample:  During the 1960s the oryx a symbol of the Arabian Peninsula were annihilated by hunters. 
http://t.co/yangEQBUQW http://t.co/jQ2eH5KGLt
After Embedding:...................


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02353205,  0.02448248, -0.03540494, ..., -0.03201516,
         -0.03076301,  0.01139504],
        [-0.0377066 ,  0.04873048,  0.01144905, ...,  0.01708207,
          0.04504705,  0.03640207],
        [ 0.03708649,  0.02676511, -0.04036485, ..., -0.0344591 ,
          0.0198534 , -0.01765348],
        ...,
        [ 0.03790363, -0.01343756, -0.00988911, ...,  0.02938997,
          0.00128095,  0.02626581],
        [ 0.03716931, -0.04934806, -0.01704571, ..., -0.03782755,
          0.01202178,  0.04536936],
        [-0.04015319, -0.00241641, -0.00853026, ...,  0.03507466,
          0.01906059,  0.02923665]]], dtype=float32)>

In [30]:
# checkout single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02353205,  0.02448248, -0.03540494,  0.03308271,  0.00352242,
        -0.00826705, -0.02217305, -0.0296743 , -0.00562846, -0.02890978,
        -0.04704982,  0.02317189,  0.01329165,  0.0202433 , -0.02097732,
        -0.04093067, -0.02591508,  0.01211145, -0.02565898,  0.03197512,
        -0.01311982,  0.04608164,  0.01881586,  0.00799065, -0.02515618,
         0.02749622, -0.00672301, -0.01587894,  0.02754419, -0.04995222,
         0.04452363, -0.01436303, -0.03833484,  0.011164  ,  0.02470087,
        -0.01114545, -0.03032132, -0.03584148,  0.02977046,  0.04901213,
        -0.0183087 , -0.04322798, -0.03080633,  0.02884339,  0.00827194,
         0.0289349 ,  0.04488733, -0.02350605, -0.0402245 ,  0.03759361,
        -0.03624244, -0.01573763, -0.04980134,  0.0167997 ,  0.01960926,
        -0.0028186 , -0.04066775,  0.01482793, -0.03528468, -0.00067971,
        -0.04329067, -0.02696383, -0.00752775,  0.0392077 , -0.04074272,
  

This is the embedding for the first word of sentence

# Model 0: creating a baseline
To create baseline, let's use Sklearn Multimodal Naive Bayes using TF-IDF formula to convert text into numbers.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# create tokenization and modeling pipeline
model_0=Pipeline([ # pipeline is like keras Sequantial here
    ('tfidf',TfidfVectorizer()), # convert words into numbers using tfidf
    ('clf',MultinomialNB()), #model the text
])
# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Evaluate baseline model

In [32]:
baseline_accuracy=model_0.score(val_sentences,val_labels)
print(f"Model 0 (Baseline) Accuracy: {baseline_accuracy*100:.3f} %")

Model 0 (Baseline) Accuracy: 79.265 %


In [33]:
baseline_preds=model_0.predict(val_sentences)
print("Predicted labels")
baseline_preds[:20]

Predicted labels


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [34]:
print("Actual labels")
val_labels[:20]

Actual labels


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [35]:
def evaluate_model(model,true_labels,predicted_labels):
  precision=tf.keras.metrics.Precision()
  precision.update_state(true_labels,predicted_labels)
  recall=tf.keras.metrics.Recall()
  recall.update_state(true_labels,predicted_labels)
  f1=(2*(precision.result()*recall.result()))/(precision.result()+recall.result())
  print(f"Precision: {precision.result()*100:.2f}%")
  print(f"Recall: {recall.result()*100:.2f}%")
  print(f"F1 score: {f1*100:.2f}%")

evaluate_model(model_0,val_labels,baseline_preds)



Precision: 88.62%
Recall: 62.64%
F1 score: 73.40%


In [36]:
# from sk-learn
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_performance_metrics(true_labels,predicted_labels):
  accuracy=accuracy_score(true_labels,predicted_labels)
  precision_score,recall_score,f1_score,_=precision_recall_fscore_support(true_labels,predicted_labels,average="weighted")
  model_results={
      "accuracy":accuracy,
      "precision":precision_score,
      "recall":recall_score,
      "f1_score":f1_score
  }
  return model_results

In [37]:
model_0_performance=calculate_performance_metrics(val_labels,baseline_preds)
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

The two evaluation methods (evaluate_model and calculate_performance_metrics) produce different results because:
* Different Libraries & Implementations

tf.keras.metrics (TensorFlow) and sklearn.metrics (scikit-learn) compute metrics differently.

Example:

>tf.keras.metrics.Precision() uses micro-averaging by default.

>sklearn.metrics.precision_recall_fscore_support() uses weighted averaging (as specified by average="weighted").
Averaging Method

* Weighted vs. Micro Averaging:

>Weighted (scikit-learn): Accounts for class imbalance by weighting metrics by class support.

>Micro (TensorFlow): Aggregates all classes equally, which can skew results if classes are imbalanced.

# Model 1: Simple dense model

In [38]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback
# create a directory to save the tensorboard logs
SAVE_DIR="mdoel_logs"

In [39]:
# Building a model with functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string") # inputs are one dimentional string
x=text_vectorizer(inputs) # turn input text into number
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # lower the dimentionality of the embedding
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [40]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_1_dense")])


Saving TensorBoard log files to: mdoel_logs/model_1_dense/20250520-144936
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.6437 - loss: 0.6495 - val_accuracy: 0.7572 - val_loss: 0.5330
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8117 - loss: 0.4586 - val_accuracy: 0.7848 - val_loss: 0.4698
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8632 - loss: 0.3549 - val_accuracy: 0.7874 - val_loss: 0.4601
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9018 - loss: 0.2848 - val_accuracy: 0.7848 - val_loss: 0.4613
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9132 - loss: 0.2365 - val_accuracy: 0.7848 - val_loss: 0.4785


In [42]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7797 - loss: 0.5103


[0.4784530699253082, 0.7847769260406494]

In [43]:
# make predictions
model_1_pred_probability=model_1.predict(val_sentences)
model_1_pred_probability.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


(762, 1)

In [44]:
model_1_pred_probability[0]

array([0.37578014], dtype=float32)

In [45]:
model_1_pred_probability=tf.squeeze(tf.round(model_1_pred_probability))
model_1_pred_probability[:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
      dtype=float32)>

In [46]:
val_labels[:50]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0])

In [47]:
model_1_perfornamce=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_1_pred_probability)
model_1_perfornamce

{'accuracy': 0.7847769028871391,
 'precision': 0.7895833902086796,
 'recall': 0.7847769028871391,
 'f1_score': 0.7817499108999565}

In [48]:
# compare with the baseline
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

Here, Our base line model outperforms the Deep Learning Model 1

# Visualizing embedding layer

In [49]:
len(words_in_vocab), words_in_vocab[:10] # the most common words

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [50]:
# get the weight matrix of embedding layer
embeded_weights=model_1.get_layer("embedding").get_weights()[0]
embeded_weights

array([[-0.07013689,  0.05173865, -0.000175  , ...,  0.01907044,
        -0.03601691, -0.07271478],
       [ 0.02961739,  0.03123991, -0.03356638, ..., -0.03936952,
         0.01307062, -0.02471191],
       [-0.04856869,  0.0584498 ,  0.0215629 , ...,  0.00828011,
         0.03495339,  0.02619412],
       ...,
       [ 0.02656636, -0.01334003,  0.02685687, ..., -0.03973215,
        -0.02910793, -0.0280315 ],
       [-0.00683283,  0.08240487,  0.03806366, ..., -0.0346344 ,
         0.00394172, -0.03554325],
       [-0.10566585,  0.09679863,  0.10368089, ..., -0.09843038,
        -0.05143359, -0.09406672]], dtype=float32)

In [51]:
embeded_weights.shape

(10000, 128)

## Embedding Projector

In [52]:
# Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format:
# a file of vectors (containing the embedding), and a file of meta data (containing the words).
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embeded_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [53]:
# If you are running this tutorial in Colaboratory, you can use the following snippet to download these files to your local machine
#  (or use the file browser, View -> Table of contents -> File browser).
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model: 2
LSTM : Long short term memory
> Input->Tokenizer->Embeddings->Layers(RNN/Dense)->Output(label probability)

In [54]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),name="input_layer",dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)
# x=layers.LSTM(units=64,return_sequences=True)(x) # when we are stacking the LSTM cells we need to set the return_sequences= true
print(x.shape)
x=layers.LSTM(64)(x)
print(x.shape)
# x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)

model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [55]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# compile the mdoel
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
# fit the model
model_2_history=model_2.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_2_lstm")])

Saving TensorBoard log files to: mdoel_logs/model_2_lstm/20250520-145015
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 46ms/step - accuracy: 0.8953 - loss: 0.2978 - val_accuracy: 0.7743 - val_loss: 0.5748
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9423 - loss: 0.1587 - val_accuracy: 0.7835 - val_loss: 0.6261
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.9502 - loss: 0.1395 - val_accuracy: 0.7808 - val_loss: 0.6393
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.9626 - loss: 0.1007 - val_accuracy: 0.7782 - val_loss: 0.7672
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9664 - loss: 0.0857 - val_accuracy: 0.7795 - val_loss: 0.8625


In [58]:
# make prediction
model_2_pred=model_2.predict(val_sentences)
model_2_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


array([[0.04924815],
       [0.7507413 ],
       [0.9996094 ],
       [0.06993864],
       [0.00113604],
       [0.9989802 ],
       [0.83910185],
       [0.9998216 ],
       [0.9997167 ],
       [0.4384557 ]], dtype=float32)

In [59]:
model_2_pred_probability=tf.squeeze(tf.round(model_2_pred))
model_2_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

## Model 2 vs Model 1 vs Model 0

Still our baseline beats our Model 1 and Model 2

In [60]:
model_2_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_2_pred_probability)
model_2_performance

{'accuracy': 0.7795275590551181,
 'precision': 0.7795261140776376,
 'recall': 0.7795275590551181,
 'f1_score': 0.7786058734339147}

In [61]:
model_1_perfornamce

{'accuracy': 0.7847769028871391,
 'precision': 0.7895833902086796,
 'recall': 0.7847769028871391,
 'f1_score': 0.7817499108999565}

In [62]:
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# Model 3: GRU
The GRU cell has similar features to an LSTM but has less parameters

In [63]:
# Build an RNN using GRU cell
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GRU(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")

In [64]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_3_history=model_3.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_3_GRU")])

Saving TensorBoard log files to: mdoel_logs/model_3_GRU/20250520-145115
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.8955 - loss: 0.2623 - val_accuracy: 0.7782 - val_loss: 0.8748
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9692 - loss: 0.0852 - val_accuracy: 0.7769 - val_loss: 0.8256
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9720 - loss: 0.0741 - val_accuracy: 0.7769 - val_loss: 0.9171
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.9758 - loss: 0.0604 - val_accuracy: 0.7808 - val_loss: 1.0805
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.9815 - loss: 0.0499 - val_accuracy: 0.7730 - val_loss: 1.0460


In [66]:
model_3_pred=model_3.predict(val_sentences)
model_3_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


array([[3.8970330e-01],
       [9.1075635e-01],
       [9.9979925e-01],
       [2.0094718e-01],
       [2.8649333e-04],
       [9.9955356e-01],
       [8.1736553e-01],
       [9.9992973e-01],
       [9.9984479e-01],
       [9.5366400e-01]], dtype=float32)

In [67]:
model_3_pred_probability=tf.squeeze(tf.round(model_3_pred))
model_3_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
model_3_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_3_pred_probability)
model_3_performance

{'accuracy': 0.7729658792650919,
 'precision': 0.7728267232157443,
 'recall': 0.7729658792650919,
 'f1_score': 0.7728842723015383}

In [69]:
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# Model 4

Normal RNN's go from the left to right but a bidirectional RNN's goes from the left to right as well as right to left.

In [70]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")


In [71]:
# summary
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [73]:
# fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model 4 bidirectional")])

Saving TensorBoard log files to: mdoel_logs/model 4 bidirectional/20250520-145210
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 29s 75ms/step - accuracy: 0.9399 - loss: 0.1892 - val_accuracy: 0.7703 - val_loss: 0.8993
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - accuracy: 0.9766 - loss: 0.0562 - val_accuracy: 0.7677 - val_loss: 1.1261
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.9760 - loss: 0.0536 - val_accuracy: 0.7690 - val_loss: 1.3258
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.9813 - loss: 0.0435 - val_accuracy: 0.7717 - val_loss: 1.3807
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - accuracy: 0.9828 - loss: 0.0379 - val_accuracy: 0.7730 - val_loss: 1.3741


In [74]:
#make prediction on bodirectional RNN
model_4_pred=model_4.predict(val_sentences)
model_4_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step


array([[2.3215036e-03],
       [6.8695492e-01],
       [9.9998456e-01],
       [1.8725333e-01],
       [1.6129610e-05],
       [9.9968034e-01],
       [8.5362875e-01],
       [9.9999446e-01],
       [9.9999034e-01],
       [3.5784280e-01]], dtype=float32)

In [75]:
# converting pred prob to the pred labels
model_4_preds=tf.squeeze(tf.round(model_4_pred))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [76]:
# calculating the metrics
model_4_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_4_preds)
model_4_performance

{'accuracy': 0.7729658792650919,
 'precision': 0.7759760952910788,
 'recall': 0.7729658792650919,
 'f1_score': 0.7702964374538457}

# CNN for the text.
we have used CNN for the images which is 2D but our text data is 1D

Inputs-> Tokenozation-> Embedding-> layer(s) (typically conv1D or pooling layers) -> outputs (class probalilities)

# Model 5 : Conv1D

In [77]:
from tensorflow.keras import layers
embedding_test=embedding(text_vectorizer(["hi there, how are you"]))
conv1d=layers.Conv1D(filters=32,kernel_size=5,padding="valid",activation="relu")
conv1d_output=conv1d(embedding_test)
max_pool=layers.GlobalAveragePooling1D()
max_pool_output=max_pool(conv1d_output)
max_pool_output=max_pool(conv1d_output)
embedding_test.shape,conv1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [78]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0459085 , -0.01117932,  0.02250412, ..., -0.00377497,
          0.0190103 ,  0.03368849],
        [-0.04698228,  0.07356405,  0.06337465, ..., -0.02753972,
         -0.06802091, -0.053634  ],
        [-0.04514555,  0.10509831, -0.01049523, ..., -0.04205988,
         -0.0300069 , -0.07922828],
        ...,
        [-0.03446162,  0.05585424, -0.02108303, ...,  0.00655703,
         -0.03994425, -0.05454998],
        [-0.03446162,  0.05585424, -0.02108303, ...,  0.00655703,
         -0.03994425, -0.05454998],
        [-0.03446162,  0.05585424, -0.02108303, ...,  0.00655703,
         -0.03994425, -0.05454998]]], dtype=float32)>

In [79]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name='model_5_conv1d')
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5.summary()

Model: "model_5_conv1d"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 11, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
model_5_history= model_5.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="Conv1D")]
                            )

Saving TensorBoard log files to: mdoel_logs/Conv1D/20250520-145339
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - accuracy: 0.9529 - loss: 0.1785 - val_accuracy: 0.7756 - val_loss: 0.9127
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9728 - loss: 0.0746 - val_accuracy: 0.7638 - val_loss: 1.0841
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.9790 - loss: 0.0548 - val_accuracy: 0.7572 - val_loss: 1.1896
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9790 - loss: 0.0520 - val_accuracy: 0.7598 - val_loss: 1.1980
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.9794 - loss: 0.0495 - val_accuracy: 0.7572 - val_loss: 1.2866


In [81]:
model_5_pred=model_5.predict(val_sentences)
model_5_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


array([[2.3333867e-01],
       [9.0633374e-01],
       [9.9978650e-01],
       [5.2296545e-02],
       [3.7686508e-08],
       [9.8743433e-01],
       [7.8062952e-01],
       [9.9995929e-01],
       [9.9999940e-01],
       [8.2665324e-01]], dtype=float32)

In [82]:
model_5_pred_probability=tf.squeeze(tf.round(model_5_pred))
model_5_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
model_5_performance=calculate_performance_metrics(val_labels,model_5_pred_probability)
model_5_performance


{'accuracy': 0.7572178477690289,
 'precision': 0.7577805667624024,
 'recall': 0.7572178477690289,
 'f1_score': 0.7554724830368381}

In [84]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") #Universal sentense encoder

In [85]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model

# Wrap USE inside a custom Keras Layer
class USEEmbedding(tf.keras.layers.Layer):
    def __init__(self):
        super(USEEmbedding, self).__init__()
        self.use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    def call(self, inputs):
        return self.use(inputs)

# Keras Input (1D: batch of strings)
input_text = tf.keras.Input(shape=(), dtype=tf.string, name="input_text")

# Embed using our wrapped USE Layer
embedding = USEEmbedding()(input_text)

# Add classification layers
x = layers.Dense(64, activation="relu")(embedding)
output = layers.Dense(1, activation="sigmoid")(x)

# Build model
model_6 = Model(inputs=input_text, outputs=output)
model_6.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Show model summary
model_6.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_text (InputLayer)         │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ use_embedding_2 (USEEmbedding)  │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,897 (128.50 KB)

 Trainable params: 32,897 (128.50 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
mdoel_6_history=model_6.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,"model_6_transfer_learning")])

Saving TensorBoard log files to: mdoel_logs/model_6_transfer_learning/20250520-145506
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.7381 - loss: 0.5813 - val_accuracy: 0.7940 - val_loss: 0.4483
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.8072 - loss: 0.4214 - val_accuracy: 0.8045 - val_loss: 0.4392
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.8265 - loss: 0.3893 - val_accuracy: 0.8084 - val_loss: 0.4329
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.8191 - loss: 0.4037 - val_accuracy: 0.8110 - val_loss: 0.4334
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8276 - loss: 0.3842 - val_accuracy: 0.8123 - val_loss: 0.4280


In [88]:
# make prediction
model_6_pred=model_6.predict(val_sentences)
model_6_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


array([[0.17949788],
       [0.810923  ],
       [0.99141896],
       [0.2182848 ],
       [0.76149297],
       [0.74641526],
       [0.98545873],
       [0.9817693 ],
       [0.9506902 ],
       [0.10581513]], dtype=float32)

In [89]:
# convrting the preds into probabilities
model_6_pred_probability=tf.squeeze(tf.round(model_6_pred))
model_6_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [90]:
model_6_results=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_6_pred_probability)
model_6_results

{'accuracy': 0.8123359580052494,
 'precision': 0.8131027998205999,
 'recall': 0.8123359580052494,
 'f1_score': 0.8113480882593861}